In [186]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import plotly.express as px
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [65]:
sales_2022 = pd.read_csv("./Data/Iowa_Liquor_Sales_2022.csv")

In [366]:
sales_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2564565 entries, 0 to 2564564
Data columns (total 29 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   Invoice/Item Number                 object 
 1   Date                                object 
 2   Store Number                        int64  
 3   Store Name                          object 
 4   Address                             object 
 5   City                                object 
 6   Zip Code                            float64
 7   Store Location                      object 
 8   County Number                       float64
 9   County                              object 
 10  Category                            int64  
 11  Category Name                       object 
 12  Vendor Number                       int64  
 13  Vendor Name                         object 
 14  Item Number                         int64  
 15  Item Description                    object 
 16  

In [367]:
sales_2022_filtered = sales_2022[['Vendor Number', 'Category Name', 'County Number', 'Volume Sold (Liters)', 'Bottles Sold', 'Sale (Dollars)']]
sales_2022_agg = sales_2022_filtered.groupby(['Vendor Number', 'Category Name', 'County Number']).agg(
    cumulative_vol = pd.NamedAgg(column='Volume Sold (Liters)', aggfunc='sum')
).reset_index(drop=False)

In [369]:
sales_2022_agg['Agg_Name'] = sales_2022_agg['Category Name'] + "_" + sales_2022_agg['County Number'].astype(str)
table = pd.pivot_table(sales_2022_agg, values='cumulative_vol', index=['Vendor Number'],
                       columns=['Agg_Name'], aggfunc="sum", fill_value=0)
table = table.reset_index(drop=False).set_index(['Vendor Number'])

imp_categories = list(sales_2022_agg[sales_2022_agg['Vendor Number'] == 35]['Agg_Name'].unique())
table = table[imp_categories].reset_index(drop=False)
table

Agg_Name,Vendor Number,100% AGAVE TEQUILA_1.0,100% AGAVE TEQUILA_2.0,100% AGAVE TEQUILA_3.0,100% AGAVE TEQUILA_4.0,100% AGAVE TEQUILA_5.0,100% AGAVE TEQUILA_6.0,100% AGAVE TEQUILA_7.0,100% AGAVE TEQUILA_8.0,100% AGAVE TEQUILA_9.0,...,WHITE RUM_90.0,WHITE RUM_91.0,WHITE RUM_92.0,WHITE RUM_93.0,WHITE RUM_94.0,WHITE RUM_95.0,WHITE RUM_96.0,WHITE RUM_97.0,WHITE RUM_98.0,WHITE RUM_99.0
0,35,44.97,9.0,163.86,124.50,8.25,91.37,5149.79,200.99,187.86,...,1282.48,923.36,1235.49,43.87,1595.21,409.74,618.97,6094.1,186.45,647.86
1,65,36.00,2.5,10.50,42.25,7.50,64.20,1575.19,185.15,235.65,...,9.00,0.00,0.00,0.00,9.00,0.00,9.00,171.0,0.00,0.00
2,79,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00
3,85,0.00,0.0,9.00,15.00,0.00,0.75,513.85,9.00,13.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00
4,86,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,888,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00
203,892,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00
204,962,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00
205,971,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00


In [373]:
scaler = MinMaxScaler()
X = scaler.fit_transform(np.array(table.drop(['Vendor Number'], axis=1)))

tsne = TSNE(n_components=2, perplexity=15)
embed_fit = tsne.fit_transform(X)

names = list(table['Vendor Number'])

history = []
for i in range(20):
    kmeans = KMeans(n_clusters=i+1, random_state=0, n_init='auto').fit(embed_fit)
    labels = kmeans.labels_
    inertia = kmeans.inertia_
    history.append([(i+1), labels, inertia])

history = pd.DataFrame(history, columns=['n_clusters', 'labels', 'inertia'])
history

,n_clusters,labels,inertia
0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",38715.250000
1,2,"[0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, ...",25359.750000
2,3,"[0, 2, 0, 2, 2, 0, 0, 2, 2, 1, 2, 2, 2, 1, 0, ...",18902.873047
3,4,"[0, 2, 0, 2, 2, 2, 0, 2, 2, 1, 0, 2, 2, 1, 0, ...",16381.331055
4,5,"[0, 2, 0, 2, 2, 0, 0, 2, 2, 1, 4, 0, 4, 4, 0, ...",12049.518555
5,6,"[0, 2, 0, 2, 2, 0, 5, 5, 2, 1, 5, 2, 5, 5, 0, ...",9669.011719
6,7,"[0, 2, 0, 2, 2, 6, 0, 2, 2, 1, 5, 0, 5, 5, 0, ...",8340.337891
7,8,"[0, 2, 0, 2, 2, 7, 0, 4, 2, 1, 4, 7, 4, 4, 0, ...",6910.688477
8,9,"[0, 2, 0, 2, 2, 7, 0, 4, 2, 8, 4, 7, 4, 5, 0, ...",5814.857422
9,10,"[0, 2, 0, 2, 2, 7, 4, 1, 2, 8, 4, 7, 4, 4, 0, ...",5385.601074


In [374]:
px.line(history, x='n_clusters', y='inertia')
# Looks like 6 clusters forms an elbow.

In [376]:
labels = history.loc[history['n_clusters'] == 6]['labels'].values[0]

data_labels = []
try:
    for i in range(len(names)):
        data_labels.append([names[i], labels[i], embed_fit[i][0], embed_fit[i][1], embed_fit[i][2]])

    data_labels = pd.DataFrame(data_labels, columns=['Brand', 'label', "X", "Y", "Z"])
except IndexError:
    for i in range(len(names)):
        data_labels.append([names[i], labels[i], embed_fit[i][0], embed_fit[i][1]])

    data_labels = pd.DataFrame(data_labels, columns=['Brand', 'label', "X", "Y"])

data_labels

,Brand,label,X,Y
0,35,0,13.096414,-11.158235
1,65,2,9.571241,13.767727
2,79,0,7.180951,-11.852859
3,85,2,7.223012,13.526455
4,86,2,-0.981392,8.943797
...,...,...,...,...
202,888,5,-3.376833,-3.281034
203,892,3,62.441910,-14.279376
204,962,1,-24.810898,2.459368
205,971,4,-6.952044,-7.169895


In [377]:
n = int(data_labels.loc[data_labels['Brand'] == 35]['label'].values)
display(data_labels.loc[data_labels['Brand'] == 35])
display(data_labels.loc[data_labels['label'] == n])

,Brand,label,X,Y
0,35,0,13.096414,-11.158235


,Brand,label,X,Y
0,35,0,13.096414,-11.158235
2,79,0,7.180951,-11.852859
5,90,0,21.180414,2.278597
14,121,0,4.686493,-10.897576
18,154,0,11.193355,-10.645796
30,214,0,10.122962,-5.547506
33,217,0,16.946339,-1.895073
34,229,0,6.917409,-14.306814
36,232,0,5.341869,-10.057987
38,239,0,18.927227,-1.875926


In [378]:
try:
    fig = px.scatter_3d(data_labels, x="X", y="Y", z='Z', color='label', hover_data=['Brand'])
except ValueError:
    fig = px.scatter(data_labels, x="X", y="Y", color='label', hover_data=['Brand'])

fig.show()